In [1]:
import os, sys
sys.path.append('.')

from dataclasses import dataclass
from pathlib import Path

import draccus
import torch
import torch.distributed as dist
import tqdm
import wandb
from accelerate import PartialState
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from transformers.modeling_outputs import CausalLMOutputWithPast
from torch.optim.lr_scheduler import StepLR
from transformers import get_linear_schedule_with_warmup

from vla.base_prompter import PurePromptBuilder
from vla.utils import PaddedCollatorForPosePrediction, runningLoss
from vla.action_tokenizer import RLbenchPoseTokenizer
from vla.dataset import RLbenchCotDataset
import numpy as np
import torch.nn.functional as F

import numpy as np
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.arm_action_modes import ArmActionMode, JointVelocity, JointPosition, EndEffectorPoseViaPlanning, EndEffectorPoseViaIK


from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.environment import Environment
from rlbench.observation_config import ObservationConfig, CameraConfig
# from rlbench.tasks.pick_described_object import PickDescribedObject
from rlbench.tasks import PutGroceriesInCupboard, PickAndLift, StackBlocks, PlaceHangerOnRack, PickDescribedObject, TakeLidOffSaucepan, SetTheTable, PutGroceriesInCupboard
from scipy.spatial.transform import Rotation as R
from matplotlib import pyplot as plt
from PIL import Image
from pyrep.const import RenderMode

2024-08-02 22:24:51.400979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 22:24:51.401082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 22:24:51.462933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 22:24:51.579911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-02 22:24:52.945930: W tensorflow/compiler/tf2

In [2]:
base_model_path = "/media/lawrence/Work/checkpoints/ecot-openvla-7b-bridge"
adapter_path = "adapter-tmp/0+weighted+pick_described_object+e1+b8+lr-0.0005+lora-r16+dropout-0.0+q-4bit"
adapter_path1 = "adapter-tmp/weighted_loss_cot_1+nll+pick_described_object2+e1+b8+lr-0.0001+lora-r16+dropout-0.0+q-4bit"
# save_path = "/media/lawrence/Work/checkpoints/vla-rl-ecot"

In [3]:
dataset_statistics: tuple = (np.array([-0.2, -0.35,  0.75199986, -np.pi/2, -np.pi/2, -np.pi/2,  0. ]), np.array([0.5, 0.35, 1.3, np.pi/2, 0, np.pi/2, 1.]))

In [4]:
processor = AutoProcessor.from_pretrained(base_model_path, trust_remote_code=True)
add_tokens = ['<g>', '</g>'] + [f'<item_{i}>' for i in np.arange(5)] + ['<o>', '</o>', '<t>', '</t>'] + [f'<stage_{i}>' for i in np.arange(2)] + ['<a>', '</a>']
processor.tokenizer.add_tokens(add_tokens)
action_tokenizer = RLbenchPoseTokenizer(processor.tokenizer,dataset_statistics)

In [5]:
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_quant_type="nf4", #llm_int8_skip_modules = ['projector'],
        )
vla = AutoModelForVision2Seq.from_pretrained(
        base_model_path,
        torch_dtype=torch.bfloat16,
        attn_implementation="sdpa",
        quantization_config=quantization_config,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        device_map = "cuda"
    )


Loading checkpoint shards: 100%|██████████| 3/3 [00:35<00:00, 11.71s/it]


In [6]:
vla.load_adapter(adapter_path,adapter_name = "adapter")
vla.load_adapter(adapter_path1,adapter_name = "adapter1")

In [7]:
vla.set_adapter("adapter")
vla.enable_adapters()

In [5]:
camera = CameraConfig(image_size=(224, 224), depth=False, point_cloud=False, mask=False)
obs_config = ObservationConfig(left_shoulder_camera=camera, right_shoulder_camera=camera, front_camera=camera, overhead_camera=camera)
obs_config.front_camera.render_mode = RenderMode.OPENGL

env = Environment(
    action_mode=MoveArmThenGripper(
        arm_action_mode=EndEffectorPoseViaPlanning(absolute_mode=True, collision_checking=False), gripper_action_mode=Discrete()),
    obs_config=obs_config,
    headless=False)
env.launch()

In [6]:
task = env.get_task(PickDescribedObject)

In [7]:
def get_data(task, variation_num):
    obs = task._scene.get_observation()
    img = Image.fromarray(obs.front_rgb,'RGB')
    gripper_pose = obs.gripper_pose
    gripper_open = obs.gripper_open
    object_pos = task._task.get_graspable_objects()[variation_num].get_position()
    target_pos = task._task.dropin_box.get_position()
    return img, gripper_pose, gripper_open, object_pos, target_pos


In [8]:
def get_instruct_prompt(gripper, instruction: str):
    prompt = f"""In: What the next key pose of gripper should the robot take to {instruction}? Current pose is <g>{gripper} </g>, let's think step by step.
    Out: """        
    return prompt

In [62]:
# task = env.get_task(PickDescribedObject)
descriptions, obs = task.reset()
img = Image.fromarray(obs.front_rgb)

descriptions

Could not find a valid joint configuration for desired end effector pose.


['put the chocolate jello powder in the basket',
 'pick up the chocolate jello powder and place it in the basket',
 'store the chocolate jello powder in the basket',
 'move the chocolate jello powder to the basket',
 'place the chocolate jello powder in the basket']

In [9]:
task = env.get_task(PickDescribedObject)
descriptions, obs = task.reset()
instr = descriptions[1]
prompt = get_instruct_prompt(instr)
image = Image.fromarray(obs.front_rgb)
inputs = processor(prompt, image).to(vla.device, dtype=torch.bfloat16)
while True:
    output_dict = vla.generate(**inputs, max_new_tokens = 50, output_scores = True, return_dict_in_generate=True, do_sample = False)
    gripper_mask, item_mask, object_mask, target_mask, stage_mask, action_mask = action_tokenizer.get_mask(output_dict.sequences)
    print(processor.tokenizer.decode(output_dict.sequences[0]))
    if action_mask.sum().item() != 7:
        print("Action mask is not correct")
        continue
    break
print(processor.tokenizer.decode(output_dict.sequences[0]))
output_logits = torch.stack(output_dict.scores, dim = 1)
action_logits = output_logits[action_mask[:,-output_logits.size(1):]][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(1,-1,action_tokenizer.n_bins)
action = action_tokenizer.get_action(action_logits)


<s> In: What the next key pose of gripper should the robot take to pick up the chocolate jello and place in the basket? Let's think step by step. 
    Out:<g>編민克ỹΈശ给</g>,<item_0>,<o>任민関</o>,<t>交ペམ</t>,<stage_0>,<a>任민共면ḳ彦弘</a></s>
<s> In: What the next key pose of gripper should the robot take to pick up the chocolate jello and place in the basket? Let's think step by step. 
    Out:<g>編민克ỹΈശ给</g>,<item_0>,<o>任민関</o>,<t>交ペམ</t>,<stage_0>,<a>任민共면ḳ彦弘</a></s>


In [10]:
class Agent(object):
    def __init__(self, vla, processor, action_tokenizer):
        self.vla = vla
        self.processor = processor
        self.action_tokenizer = action_tokenizer

    def get_openvla_prompt(self, instruction: str):
        SYSTEM_PROMPT = "You are an assistant helping to control a robotic manipulator. The robot performs tasks by following a series of steps to interact with objects in its environment. The environment includes items like soup cans and baskets, and the robot uses a gripper to pick up and move these items.\n\nInstructions format:\n- 'USER': Describes the task to be performed.\n- 'ASSISTANT': Provides a detailed step-by-step plan for the robot to execute the task.\n\nThe 'ASSISTANT' response includes:\n1. A logical step-by-step plan for the task.\n2. The current positions of relevant objects and the gripper.\n3. The current state of the gripper (whether it has grasped the object or not).\n4. The next key pose of the gripper to achieve the task.\n\nExample:\n\nUSER: What action should the robot take to pick up the soup and place it in the basket?\nASSISTANT: Let's think step by step. The plan is to move the gripper to the soup and pick it up, then move over the basket, and then place the soup in the basket. The soup is located at <object>ĉ‖호 </object>. The basket is located at <target>Ζ‖ご </target>. The gripper pose is <gripper>阳‖素군雅导弘 </gripper>. The gripper hasn't grasped the soup. So the current step is to move the gripper to the soup and pick it up. The next key pose of the gripper is <action>机‖素秀麻방弘 </action>. \n <current conversation>"
        return f"{SYSTEM_PROMPT} USER: What action should the robot take to {instruction.lower()}? ASSISTANT: Let's think step by step,"

    def get_instruct_prompt(self, instruction: str):
        prompt = f"""In: What the next key pose of gripper should the robot take to {instruction}? Let's think step by step. 
        Out: <g>"""        
        return prompt

    def act(self, obs, instr):
        prompt = self.get_instruct_prompt(instr)
        image = Image.fromarray(obs.front_rgb)
        inputs = self.processor(prompt, image).to(self.vla.device, dtype=torch.bfloat16)
        while True:
            output_dict = vla.generate(**inputs, max_new_tokens = 50, output_scores = True, return_dict_in_generate=True, do_sample = True, temperature = 0.5)
            gripper_mask, item_mask, object_mask, target_mask, stage_mask, action_mask = action_tokenizer.get_mask(output_dict.sequences)
            if action_mask.sum().item() != 7:
                print("Action mask is not correct")
                continue
            break
        print(processor.tokenizer.decode(output_dict.sequences[0]))
        output_logits = torch.stack(output_dict.scores, dim = 1)
        action_logits = output_logits[action_mask[:,-output_logits.size(1):]][:,action_tokenizer.action_token_begin_idx:processor.tokenizer.vocab_size].view(1,-1,action_tokenizer.n_bins)
        action = action_tokenizer.get_action(action_logits)

        return action
        

In [11]:
agent = Agent(vla, processor, action_tokenizer)

In [12]:
def randomize_pose(task):
    np.random.seed()
    while True:
        try:
            pos_lower_bound = np.array([-0.2, -0.35, task._scene._workspace_minz])
            pos_upper_bound = np.array([task._scene._workspace_maxx, 0.35, 1.3])
            rot_lower_bound = np.array([-np.pi/2, 0, -np.pi/2])
            rot_upper_bound = np.array([np.pi/2, np.pi/2, np.pi/2])
            pos = np.random.uniform(pos_lower_bound, pos_upper_bound)
            euler = np.random.uniform(rot_lower_bound, rot_upper_bound)
            euler[0] = np.clip(np.random.normal(0, np.pi/6),-np.pi/2,np.pi/2)
            euler[1] = -np.clip(abs(np.random.normal(0,np.pi/6)),0, np.pi/2)
            trans = lambda rx: rx - np.pi if rx > 0 else rx + np.pi 
            euler[0] = trans(euler[0])
            quat = R.from_euler('xyz', euler).as_quat()
            joint_position = task._scene.robot.arm.solve_ik_via_sampling(position=pos, euler=euler,trials=10)
            task.step(np.concatenate([pos, quat, [1]]))
            break
        except Exception as e:
            print(e)
            continue
    return task._scene.get_observation()

In [ ]:
task = env.get_task(PickDescribedObject)
task.set_variation(2)
training_steps = 1000
episode_length = 100
for i in range(training_steps):
    if i % episode_length == 0:
        print('Reset Episode')
        descriptions, obs = task.reset()
        obs = randomize_pose(task)
        print(descriptions[1])   
    plt.imshow(obs.front_rgb)     
    plt.show()    
    action = agent.act(obs,descriptions[1])
    action_rotation = R.from_euler('xyz', action[3:6])
    action_quaternion = action_rotation.as_quat()
    # print(delta_quaternion)  # returns (qx, qy, qz, qw)
    action = np.concatenate([action[0:3], action_quaternion, action[-1:]])
    print(action)
    obs, reward, terminate = task.step(action)
    print(reward)

In [ ]:
task = env.get_task(PickDescribedObject)
lower_bound = np.array([task._scene._workspace_minx, task._scene._workspace_miny, task._scene._workspace_minz])
upper_bound = np.array([task._scene._workspace_maxx, task._scene._workspace_maxy, task._scene._workspace_maxz])
training_steps = 1000
episode_length = 100
obs = None
while 1:
    try:
        for i in range(training_steps):
            if i % episode_length == 0:
                print('Reset Episode')
                descriptions, obs = task.reset()
                plt.imshow(obs.front_rgb)
                print(descriptions[1])            
                pos = np.random.uniform(lower_bound, upper_bound)
                rot = np.random.uniform(-np.pi, np.pi, 3)
                joint_position = task._scene.robot.arm.solve_ik_via_sampling(position=pos, euler=rot)
                task._scene.robot.arm.set_joint_positions(joint_position[0], disable_dynamics=True)
            action = agent.act(obs,descriptions[1])
            action_rotation = R.from_euler('xyz', action[3:6])
            action_quaternion = action_rotation.as_quat()
            # print(delta_quaternion)  # returns (qx, qy, qz, qw)
            action = np.concatenate([action[0:3], action_quaternion, action[-1:]])
            print(action)
            obs, reward, terminate = task.step(action)
            print(reward)
    except Exception as e:
        print(e)


Reset Episode
pick up the chocolate jello and place in the basket
Could not find a valid joint configuration for desired end effector pose.
Reset Episode
pick up the chocolate jello and place in the basket
Could not find a valid joint configuration for desired end effector pose.
Reset Episode
pick up the chocolate jello and place in the basket
Could not find a valid joint configuration for desired end effector pose.
Reset Episode
pick up the chocolate jello and place in the basket


/home/lawrence/anaconda3/envs/VLA-RL/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/lawrence/anaconda3/envs/VLA-RL/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


<s> In: You are an assistant helping to control a robotic manipulator. The robot performs tasks by following a series of steps to interact with objects in its environment. What the next key pose of gripper should the robot take to pick up the chocolate jello and place in the basket? Out: Let's think step by step, First, move the gripper to the chocolate jello and pick it up. Next, move over the basket, and finally, place the chocolate jello in the basket. The chocolate jello is located at <object>Ġ瀬합ữrivialŸ</s>
Expected `angles` to be at most 2-dimensional with width equal to number of axes specified, got (0,) for shape
Reset Episode
pick up the chocolate jello and place in the basket
<s> In: You are an assistant helping to control a robotic manipulator. The robot performs tasks by following a series of steps to interact with objects in its environment. What the next key pose of gripper should the robot take to pick up the chocolate jello and place in the basket? Out: Let's think step